# 30-Day Scheduler Sync

This notebook syncs event data from a Google Sheet to a Google Calendar. It includes features for overlap detection, exponential backoff for API rate limiting, and incremental saving of created events.

## Step 1: Setup and Authentication

In [1]:
import os
import sys
import pandas as pd
import datetime

# Add src to path so we can import our modules
sys.path.append('src')

from sync_scheduler import get_google_services, prepare_event_body, create_event

# --- Configuration ---
SPREADSHEET_ID = '10Z993MrZHH0Da_pXEFZoo0MBdxKhf619fZSuuvaAdlQ'
SIGNUP_SHEET = 'Signup'
CONTACT_SHEET = 'Teacher Contact'
CALENDAR_ID = 'b45a2d5121fed950d815cfa167dd4b3a6aa74c5d62fea928702e6f4300d96545@group.calendar.google.com'
TEMPLATE_FILE = '_calendar_event_template.jsonc'
CREATED_EVENTS_CSV = 'logs/created_events.csv'
SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets.readonly', 
    'https://www.googleapis.com/auth/calendar.events'
]

sheets_service, calendar_service = get_google_services()
print("Services initialized.")

Services initialized.


## Step 2: Fetch Event Data from Google Sheets

In [2]:
from google_sheets_data import fetch_google_sheets_data

# Fetch data using the external module
df_pending, teacher_map = fetch_google_sheets_data(
    sheets_service=sheets_service,
    spreadsheet_id=SPREADSHEET_ID,
    signup_sheet=SIGNUP_SHEET,
    contact_sheet=CONTACT_SHEET,
    template_file=TEMPLATE_FILE
)
display(df_pending.head())

Loaded 14 teacher contacts.
Found 144 events in Google Sheets.


,Summary,Begin,Teacher,Contact,Date,Day,Start
0,10-Minute Guided Session: Stephen Holsenbeck,2026-01-01T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-01-01,Thursday,07:00
1,10-Minute Guided Session: Noah Binder,2026-01-01T09:00:00,Noah Binder,"{'First Name': 'Noah', 'Last Name': 'Binder', ...",2026-01-01,Thursday,09:00
2,10-Minute Guided Session: Eileen Knott,2026-01-01T20:00:00,Eileen Knott,"{'First Name': 'Eileen', 'Last Name': 'Knott',...",2026-01-01,Thursday,20:00
3,10-Minute Guided Session: Noah Binder,2026-01-01T21:00:00,Noah Binder,"{'First Name': 'Noah', 'Last Name': 'Binder', ...",2026-01-01,Thursday,21:00
4,10-Minute Guided Session: Noah Binder,2026-01-02T08:00:00,Noah Binder,"{'First Name': 'Noah', 'Last Name': 'Binder', ...",2026-01-02,Friday,08:00


## Step 3: Overlap Detection

In [8]:
from overlap_detection import check_overlaps, update_created_events_csv

UPDATE_CREATED_EVENTS = True # Set to True to refresh from Calendar

if UPDATE_CREATED_EVENTS:
    update_created_events_csv(calendar_service, CALENDAR_ID, CREATED_EVENTS_CSV)

if os.path.exists(CREATED_EVENTS_CSV):
    df_created = pd.read_csv(CREATED_EVENTS_CSV)
    print(f"Loaded {len(df_created)} already created events.")
    
    # Mark overlaps using the new function
    df_pending['is_overlap'] = check_overlaps(df_pending, df_created)
    
    overlaps = df_pending[df_pending['is_overlap'] == True]
    print(f"Detected {len(overlaps)} overlapping events.")
    display(overlaps)
else:
    df_pending['is_overlap'] = False
    print("No created_events.csv found. Starting fresh.")

Syncing logs/created_events.csv with Google Calendar...
Successfully synced 250 events to logs/created_events.csv.
Loaded 250 already created events.
Detected 144 overlapping events.


,Summary,Begin,Teacher,Contact,Date,Day,Start,is_overlap
0,10-Minute Guided Session: Stephen Holsenbeck,2026-01-01T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-01-01,Thursday,07:00,True
1,10-Minute Guided Session: Noah Binder,2026-01-01T09:00:00,Noah Binder,"{'First Name': 'Noah', 'Last Name': 'Binder', ...",2026-01-01,Thursday,09:00,True
2,10-Minute Guided Session: Eileen Knott,2026-01-01T20:00:00,Eileen Knott,"{'First Name': 'Eileen', 'Last Name': 'Knott',...",2026-01-01,Thursday,20:00,True
3,10-Minute Guided Session: Noah Binder,2026-01-01T21:00:00,Noah Binder,"{'First Name': 'Noah', 'Last Name': 'Binder', ...",2026-01-01,Thursday,21:00,True
4,10-Minute Guided Session: Noah Binder,2026-01-02T08:00:00,Noah Binder,"{'First Name': 'Noah', 'Last Name': 'Binder', ...",2026-01-02,Friday,08:00,True
...,...,...,...,...,...,...,...,...
139,10-Minute Guided Session: Brad Constable,2026-01-31T08:00:00,Brad Constable,"{'First Name': 'Brad', 'Last Name': 'Constable...",2026-01-31,Saturday,08:00,True
140,10-Minute Guided Session: Stephen Holsenbeck,2026-01-31T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-01-31,Saturday,07:00,True
141,10-Minute Guided Session: Mary Hill,2026-01-31T15:00:00,Mary Hill,"{'First Name': 'Mary', 'Last Name': 'Hill', 'P...",2026-01-31,Saturday,15:00,True
142,10-Minute Guided Session: Kat McLeod,2026-01-31T20:00:00,Kat McLeod,"{'First Name': 'Kat', 'Last Name': 'McLeod', '...",2026-01-31,Saturday,20:00,True


## Step 4: User Review and Pruning

In [7]:
print("Review the overlaps above.")
confirm = input("Remove overlapping events from the list to be created? (y/n): ")

if confirm.lower() == 'y':
    df_to_create = df_pending[df_pending['is_overlap'] == False].copy()
    print(f"Pruned {len(df_pending) - len(df_to_create)} events. {len(df_to_create)} remaining.")
else:
    df_to_create = df_pending.copy()
    print(f"Proceeding with all {len(df_to_create)} events.")
df_to_create

Review the overlaps above.
Pruned 70 events. 74 remaining.


,Summary,Begin,Teacher,Contact,Date,Day,Start,is_overlap
27,10-Minute Guided Session: Eileen Knott,2026-01-06T20:00:00,Eileen Knott,"{'First Name': 'Eileen', 'Last Name': 'Knott',...",2026-01-06,Tuesday,20:00,False
59,10-Minute Guided Session: Kim Weeber,2026-01-13T20:00:00,Kim Weeber,"{'First Name': 'Kim', 'Last Name': 'Weeber', '...",2026-01-13,Tuesday,20:00,False
60,10-Minute Guided Session: Rob Thompson,2026-01-13T21:00:00,Rob Thompson,"{'First Name': 'Rob', 'Last Name': 'Thompson',...",2026-01-13,Tuesday,21:00,False
61,10-Minute Guided Session: Colleen Godfrey,2026-01-14T08:00:00,Colleen Godfrey,"{'First Name': 'Colleen', 'Last Name': 'Godfre...",2026-01-14,Wednesday,08:00,False
63,10-Minute Guided Session: Colleen Godfrey,2026-01-14T12:00:00,Colleen Godfrey,"{'First Name': 'Colleen', 'Last Name': 'Godfre...",2026-01-14,Wednesday,12:00,False
...,...,...,...,...,...,...,...,...
139,10-Minute Guided Session: Brad Constable,2026-01-31T08:00:00,Brad Constable,"{'First Name': 'Brad', 'Last Name': 'Constable...",2026-01-31,Saturday,08:00,False
140,10-Minute Guided Session: Stephen Holsenbeck,2026-01-31T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-01-31,Saturday,07:00,False
141,10-Minute Guided Session: Mary Hill,2026-01-31T15:00:00,Mary Hill,"{'First Name': 'Mary', 'Last Name': 'Hill', 'P...",2026-01-31,Saturday,15:00,False
142,10-Minute Guided Session: Kat McLeod,2026-01-31T20:00:00,Kat McLeod,"{'First Name': 'Kat', 'Last Name': 'McLeod', '...",2026-01-31,Saturday,20:00,False


## Step 5: Preview and Create Events

In [ ]:
template_text = open(TEMPLATE_FILE, 'r').read()
print(f"Events to create: {len(df_to_create)}")
display(df_to_create[['Summary', 'Begin']])

run_confirm = input("Proceed with event creation? (y/n): ")

if run_confirm.lower() == 'y':
    for idx, row in df_to_create.iterrows():
        try:
            event_body = prepare_event_body(row, template_text)
            created_event = create_event(calendar_service, CALENDAR_ID, event_body)
            
            # Incremental save
            new_row = pd.DataFrame([{
                'Summary': created_event.get('summary'),
                'ID': created_event.get('id'),
                'Begin': created_event['start'].get('dateTime') or created_event['start'].get('date')
            }])
            
            if os.path.exists(CREATED_EVENTS_CSV):
                new_row.to_csv(CREATED_EVENTS_CSV, mode='a', header=False, index=False)
            else:
                new_row.to_csv(CREATED_EVENTS_CSV, index=False)
            
            print(f"Created: {created_event.get('summary')} ({created_event.get('id')})")
        except Exception as e:
            print(f"Failed to create event for {row['Summary']}: {e}")
else:
    print("Creation cancelled.")

## Step 6: Delete Events (Cleanup)

In [ ]:
if os.path.exists(CREATED_EVENTS_CSV):
    df_to_delete = pd.read_csv(CREATED_EVENTS_CSV)
    print(f"Loaded {len(df_to_delete)} events from {CREATED_EVENTS_CSV}.")
    
    confirm_del = input("DELETE all these events from Google Calendar? (y/n): ")
    if confirm_del.lower() == 'y':
        for idx, row in df_to_delete.iterrows():
            try:
                calendar_service.events().delete(calendarId=CALENDAR_ID, eventId=row['ID']).execute()
                print(f"Deleted: {row['Summary']}")
            except Exception as e:
                print(f"Failed to delete {row['ID']}: {e}")
    else:
        print("Deletion cancelled.")
else:
    print("Nothing to delete.")